In [9]:
import rasterio
import numpy as np
import cv2
import numpy as numpy
import matplotlib.pyplot as plt
from osgeo import gdal
import argparse

# %matplotlib inline

# # import xarray as xr
# import numpy as np
# import geopandas as gpd
import scipy
# import matplotlib.pyplot as plt
from osgeo import gdal
# from datacube.utils.cog import write_cog
# from datacube.utils.geometry import Geometry
from skimage.segmentation import quickshift
import rasterio
from skimage.segmentation import felzenszwalb
from skimage import img_as_float
import numpy as np
import matplotlib.pyplot as plt

In [10]:
pixel_vals  = gdal.Open(r'E:\wenqu\segmentationn\tasseled_cap_greeness\MASKED\site7_TP.tif').ReadAsArray().astype(np.float64)

In [11]:
# # Define the no-data value from the band information
# nodata_value = 0.0

# # Mask the no-data value with np.nan (or another specified value)
# pixel_vals[pixel_vals == nodata_value] = np.nan 

In [15]:
# Calculate the segments
labels = quickshift(pixel_vals,
                      kernel_size=5,
                      convert2lab=False,
                      max_dist=50,
                      ratio=1.0)

In [16]:
# Assuming 'segments' is the result from your 'quickshift' function call
unique_segments = np.unique(labels)
number_of_segments = len(unique_segments)

print("Number of segments:", number_of_segments)

Number of segments: 2354


In [6]:
# plt.imshow(segments)

In [17]:
import geopandas as gpd
from shapely.geometry import shape
import rasterio
from rasterio.features import shapes
import numpy as np

# Load the original raster image to get its transform and CRS
image_path = r'E:\wenqu\segmentationn\tasseled_cap_greeness\MASKED\site7_TP.tif'
with rasterio.open(image_path) as src:
    transform = src.transform
    crs = src.crs

# Assuming 'labels' contains the result from the Watershed segmentation

# Convert the segmented image to shapes (polygons)
mask = labels > 0  # Assuming 0 is the background or non-segmented label

# Extract polygons and values from the labeled matrix
polygons = shapes(labels.astype(np.int16), mask=mask, transform=transform)

# Prepare a list for storing geometry data
geometry_list = []

for polygon, value in polygons:
    # Convert the polygon shapes to Shapely geometry
    geom = shape(polygon)
    geometry_list.append(geom)

# Create a GeoDataFrame from the list of geometries
gdf = gpd.GeoDataFrame(geometry=geometry_list)

# Set the coordinate reference system (CRS) of the GeoDataFrame from the raster
gdf.crs = crs

# Write the GeoDataFrame to a shapefile
output_shapefile = r'E:\wenqu\segmentationn\site7_TP_segments.shp'
gdf.to_file(output_shapefile)

print(f'Shapefile saved to {output_shapefile}')



Shapefile saved to E:\wenqu\segmentationn\site7_TP_segments.shp


In [8]:
import rasterio

# Path to the existing file from which to copy the transform and CRS
input_file_path = r'E:\wenqu\segmentationn\tasseled_cap_greeness\MASKED\site6ab_TP.tif'

# Path to the new file
output_file_path = r'E:\wenqu\segmentationn\site6labels.tif'

# Open the existing file to read its spatial metadata
with rasterio.open(input_file_path) as src:
    # Extract the metadata from the source file
    out_meta = src.meta.copy()

# Update the metadata for the new file based on your array's properties
# Assume segments_zonal_mean_qs is your array to be saved
out_meta.update({
    'dtype': 'float32',  # Make sure to set this to the dtype of your segments_zonal_mean_qs array
    'count': 1,  # Set the number of bands in your output file; adjust if necessary
    'height': labels.shape[0],
    'width': labels.shape[1],
    # 'nodata': None, # Uncomment and set this if your array has a specific nodata value
})

# Write the array to a new TIFF file using the copied+updated metadata
with rasterio.open(output_file_path, 'w', **out_meta) as dst:
    dst.write(labels, 1)  # Writing your array as the first band